In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from datetime import timedelta
import numpy as np

In [2]:
# Read the CSV file and parse the dates
final_result = pd.read_csv('final_result.csv', parse_dates=False)
all_ape_data = pd.read_csv('all_ape_data.csv', parse_dates=False)
all_ape_data_nodup_rsi = pd.read_csv('all_ape_data_nodup_rsi.csv', parse_dates=False)

In [3]:
def convert_to_datetime(df, columns=None):
    """
    Convert date-like columns in a DataFrame to datetime type.

    Parameters:
    df (DataFrame): Input DataFrame.
    columns (list of str, optional): List of columns to convert. 
                                     If None, attempts to convert all object type columns.

    Returns:
    DataFrame: DataFrame with date-like columns converted to datetime type.
    """
    # Copy the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # If no specific columns are provided, select all object type columns
    if columns is None:
        columns = df_copy.columns[df_copy.dtypes == 'object']
        df[columns] = df[columns].astype(str)
    
    # Iterate through each specified column
    for col in columns:
        print(f"Attempting column: {col}")
        # Check if the column can be converted to datetime
        try:

            # date1 = pd.to_datetime(df_copy[col], errors='coerce', format='%Y-%m-%d %H:%M:%S')
            # date2 = pd.to_datetime(df_copy[col], errors='coerce', format='%Y-%m-%d')

            # df[col] = date1.fillna(date2)
            df_copy[col]  = pd.to_datetime(df_copy[col], errors='coerce', format="mixed", dayfirst=True)
            print(f"Column {col} transformed successfully.\n")
        except Exception as e:
            print(f"Column {col} is incompatible with datetime: {e}\n")
            # If conversion fails, skip to the next column
            continue
            
    return df_copy

In [4]:
all_ape_data_nodup_rsi = convert_to_datetime(all_ape_data_nodup_rsi, columns=['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'])

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ transformed successfully.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed successfully.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed successfully.



In [5]:
all_ape_data_nodup_rsi

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,RSI,RSI_ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,RSI_ΔΗΜΟΣ,RSI_ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,RSI_ΘΕΣΗ,ΔΙΑΡΚΕΙΑ
0,514,2001-02-19,ΛΑΚΜΟΣ ΕΝΕΡΓΕΙΑΚΗ ΑΕ,ΑΔ-00001,2001-05-29,2026-05-29,ΗΠΕΙΡΟΥ,ΙΩΑΝΝΙΝΩΝ,ΜΕΤΣΟΒΟΥ,ΕΓΝΑΤΙΑΣ,ΓΚΟΥΡΑ ΜΙΚΡΟ ΠΕΡΙΣΤΕΡΙ,2.155,ΜΥΗΕ,3.584713,4.546388,2.459849,5.373287,9.019446,9131.0
1,67,2001-02-13,ΑΡΑΜΠΑΤΖΗΣ Β.Γ. ΑΕ,ΑΔ-00002,2001-05-29,2026-05-29,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,ΑΛΜΩΠΙΑΣ,ΑΡΙΔΑΙΑΣ,ΤΟΥΠΛΙΤΣΑ ΠΡΟΜΑΧΩΝΑ,0.830,ΜΥΗΕ,1.191248,6.667842,8.607129,8.531909,9.019446,9131.0
2,38,2001-02-08,ΚΑΣΤΡΙ ΕΥΒΟΙΑΣ ΑΕ,ΑΔ-00022,2001-06-18,2041-03-22,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ,ΚΑΣΤΡΙ,5.000,ΑΙΟΛΙΚΑ,7.230080,5.585120,6.619634,2.944487,3.527086,14522.0
3,439,2001-02-19,ΥΔΡΟΗΛΕΚΤΡΙΚΗ ΕΥΡΥΤΑΝΙΑΣ ΑΕ,ΑΔ-00023,2001-06-18,2026-06-18,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΡΥΤΑΝΙΑΣ,ΑΓΡΑΦΩΝ,ΑΓΡΑΦΩΝ,ΜΟΝΑΣΤΗΡΑΚΙ,3.000,ΜΥΗΕ,0.614962,5.595114,4.538448,4.919698,6.313612,9131.0
4,65,2001-02-12,ΠΟΛΥΠΟΤΑΜΟΣ ΑΙΟΛΙΚΗ ΕΝΕΡΓΕΙΑ ΑΕ,ΑΔ-00024,2001-06-18,2026-06-18,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΓΚΕΡΚΙ-ΠΥΡΓΑΡΙ ΠΟΛΥΠΟΤΑΜΟΥ,12.000,ΑΙΟΛΙΚΑ,7.230080,5.585120,6.619634,6.046433,8.565779,9131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39419,Γ-06245,2017-06-12,F-GRID ΝΕΜΕΣΙΣ ΑΙΟΛΙΚΗ ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚΗ ...,ΑΔ-04523,2021-07-29,2046-07-29,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΥΜΗΣ - ΑΛΙΒΕΡΙΟΥ,"ΚΟΝΙΣΤΡΩΝ,ΤΑΜΥΝΕΩΝ",ΜΑΚΡΥΑ ΛΕΝΙΑ,25.000,ΑΙΟΛΙΚΑ,1.593403,2.256876,2.720094,1.929145,1.929145,9131.0
39420,Γ-05876,2014-09-10,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΕΜΠΟΡΙΚΗ ...,ΑΔ-04525,2021-07-29,2046-07-29,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,"ΚΥΜΗΣ - ΑΛΙΒΕΡΙΟΥ, ΚΑΡΥΣΤΟΥ",ΑΥΛΩΝΟΣ - ΔΥΣΤΥΩΝ - ΜΑΡΜΑΡΙΟΥ - ΣΤΥΡΕΩΝ,ΑΣΠΡΗ ΠΕΤΡΑ - ΠΥΡΓΑΡΙ - ΑΧΛΑΔΙΕΣ,111.500,ΑΙΟΛΙΚΑ,1.593403,2.256876,1.929145,2.893717,2.893717,9131.0
39421,Γ-05948,2015-06-05,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΕΜΠΟΡΙΚΗ ...,ΑΔ-04526,2021-07-29,2046-07-29,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΥΜΗΣ - ΑΛΙΒΕΡΙΟΥ,ΔΥΣΤΥΩΝ,ΑΓΙΑ ΤΡΙΑΣ,4.200,ΑΙΟΛΙΚΑ,1.593403,2.256876,2.720094,2.723498,0.890374,9131.0
39422,Γ-02883,2007-10-09,ΕΥΒΟΪΚΟΣ ΑΝΕΜΟΣ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ ΠΑΡΑΓΩΓΗΣ ΗΛΕ...,ΑΔ-04529,2021-07-15,2046-07-15,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΔΙΡΦΥΩΝ - ΜΕΣΣΑΠΙΩΝ,ΔΙΡΦΥΩΝ,ΨΗΛΟΣ ΒΡΑΧΟΣ ΜΕΣΣΑΠΙΩΝ,25.300,ΑΙΟΛΙΚΑ,1.593403,2.256876,2.066941,2.893717,2.893717,9131.0


In [59]:
# Aggregate MW data by month
monthly_aggregation = all_ape_data_nodup_rsi.groupby(pd.Grouper(key='ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', freq='M')).agg({
    'ΙΣΧΥΣ (MW)': ['count','sum', 'mean', 'min', 'max'],
    'RSI': ['mean', 'min', 'max'],
    'RSI_ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ': ['mean', 'min', 'max'],
    'RSI_ΔΗΜΟΣ ': ['mean', 'min', 'max'],
    'RSI_ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ': ['mean', 'min', 'max'],
    'RSI_ΘΕΣΗ': ['mean', 'min', 'max'],
})

monthly_aggregation = monthly_aggregation.dropna()

# Extend DataFrame with future dates
future_dates = pd.date_range(start=monthly_aggregation.index[-1], periods=48, freq='M') + pd.DateOffset(months=1)
future_index = pd.DatetimeIndex(future_dates)
future_data = pd.DataFrame(index=future_index)

# Concatenate current and future data
monthly_aggregation = pd.concat([monthly_aggregation, future_data])

# Flatten the multi-index columns
monthly_aggregation.columns = ['_'.join(col).strip() for col in monthly_aggregation.columns.values]

/tmp/ipykernel_20830/3859471592.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/tmp/ipykernel_20830/3859471592.py:14: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [60]:
monthly_aggregation

,ΙΣΧΥΣ (MW)_count,ΙΣΧΥΣ (MW)_sum,ΙΣΧΥΣ (MW)_mean,ΙΣΧΥΣ (MW)_min,ΙΣΧΥΣ (MW)_max,RSI_mean,RSI_min,RSI_max,RSI_ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ_mean,RSI_ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ_min,RSI_ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ_max,RSI_ΔΗΜΟΣ _mean,RSI_ΔΗΜΟΣ _min,RSI_ΔΗΜΟΣ _max,RSI_ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ_mean,RSI_ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ_min,RSI_ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ_max,RSI_ΘΕΣΗ_mean,RSI_ΘΕΣΗ_min,RSI_ΘΕΣΗ_max
1988-12-31,2.0,17.00,8.500000,8.500,8.500,0.614962,0.614962,0.614962,0.973781,0.973781,0.973781,0.943573,0.943573,0.943573,1.288492,1.288492,1.288492,9.019446,9.019446,9.019446
1990-12-31,2.0,0.33,0.165000,0.165,0.165,0.718299,0.718299,0.718299,2.121334,2.121334,2.121334,2.893717,2.893717,2.893717,2.893717,2.893717,2.893717,2.893717,2.893717,2.893717
1991-12-31,6.0,2.27,0.378333,0.300,0.450,0.478968,0.113554,0.718299,1.586117,0.568794,2.143494,4.459848,2.095450,9.019446,4.725937,2.264648,9.019446,4.397692,1.279913,9.019446
1992-12-31,14.0,47.51,3.393571,0.700,10.800,0.752677,0.605050,1.191248,2.482484,2.016114,4.040385,2.464288,2.073111,3.616935,3.276151,0.826776,9.019446,3.663932,2.159490,9.019446
1993-12-31,4.0,13.20,3.300000,0.600,6.000,1.574075,1.191248,1.956902,1.619850,1.219773,2.019928,2.394163,2.066941,2.721385,3.255486,2.130097,4.380874,5.956582,2.893717,9.019446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
class ManualSeasonalDecomposition:
    def __init__(self, data: pd.Series, period: int):
        """
        Initialize the ManualSeasonalDecomposition class.

        Parameters:
        - data (pd.Series): The input time series data.
        - period (int): The period of the seasonality (e.g., 12 for monthly data with yearly seasonality).
        """
        self.data = data
        self.period = period

    def moving_average(self, window: int) -> pd.Series:
        """
        Calculate the moving average of the time series.

        Parameters:
        - window (int): The window size for the moving average.

        Returns:
        - pd.Series: The moving average series.
        """
        return self.data.rolling(window=window, center=True).mean()

    def decompose(self) -> dict:
        """
        Perform seasonal decomposition manually.

        Returns:
        - dict: A dictionary containing trend, seasonal, and residual components.
        """
        # Estimate the trend component using a centered moving average
        trend = self.moving_average(window=self.period)
        
        # Detrend the series by subtracting the trend component
        detrended = self.data - trend
        
        # Extract the seasonal component by averaging the detrended values for each period
        seasonal = detrended.groupby(detrended.index.month).transform('mean')
        
        # Calculate the residual component
        residual = self.data - trend - seasonal
        
        return {
            'trend': trend,
            'seasonal': seasonal,
            'residual': residual
        }

In [62]:


class TimeSeriesFeatureEngineering:
    def __init__(self, data: pd.DataFrame, mw_prefix: str = '(MW)', rsi_prefix: str = 'RSI_'):
        """
        Initialize the TimeSeriesFeatureEngineering class.

        Parameters:
        - data (pd.DataFrame): The input dataframe with time series data.
        - mw_prefix (str): The prefix for MW features. Default is 'MW_'.
        - rsi_prefix (str): The prefix for RSI features. Default is 'RSI_'.
        """
        self.data = data
        self.mw_prefix = mw_prefix
        self.rsi_prefix = rsi_prefix

    def calculate_rolling_metrics(self, prefix: str, window_sizes: list = [3, 6, 7, 11, 12, 24]) -> pd.DataFrame:
        """
        Calculate rolling metrics for the features with the given prefix.

        Parameters:
        - prefix (str): The prefix to filter columns.
        - window_sizes (list): List of window sizes for rolling calculations. Default is [3, 6, 12].

        Returns:
        - pd.DataFrame: DataFrame with rolling metrics.
        """
        rolling_metrics = pd.DataFrame(index=self.data.index)
        
        for window_size in window_sizes:
            for feature in self.data.filter(like=prefix).columns:
                rolling_metrics[f'{feature}_mean_rolling_{window_size}'] = self.data[feature].rolling(window=window_size).mean()
                rolling_metrics[f'{feature}_std_rolling_{window_size}'] = self.data[feature].rolling(window=window_size).std()
                rolling_metrics[f'{feature}_min_rolling_{window_size}'] = self.data[feature].rolling(window=window_size).min()
                rolling_metrics[f'{feature}_max_rolling_{window_size}'] = self.data[feature].rolling(window=window_size).max()
                rolling_metrics[f'{feature}_skew_rolling_{window_size}'] = self.data[feature].rolling(window=window_size).skew()
                rolling_metrics[f'{feature}_ewm_{window_size}'] = self.data[feature].ewm(span=window_size).mean()
        
        return rolling_metrics

    def calculate_autocorrelation(self, prefix: str, lags: list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 24, 48]) -> pd.DataFrame:
        """
        Calculate autocorrelation for the features with the given prefix.

        Parameters:
        - prefix (str): The prefix to filter columns.
        - lags (list): List of lag values for autocorrelation calculations. Default is [1, 2, 3].

        Returns:
        - pd.DataFrame: DataFrame with autocorrelation values.
        """
        autocorr = pd.DataFrame(index=self.data.index)
        
        for lag in lags:
            for feature in self.data.filter(like=prefix).columns:
                autocorr[f'{feature}_autocorr_lag_{lag}'] = self.data[feature].autocorr(lag=lag)
        
        return autocorr

    def create_lagged_variables(self, prefix: str, lag_lengths: list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 24, 46, 58]) -> pd.DataFrame:
        """
        Create lagged variables for the features with the given prefix.

        Parameters:
        - prefix (str): The prefix to filter columns.
        - lag_lengths (list): List of lag lengths to create lagged variables. Default is [1, 2, 3].

        Returns:
        - pd.DataFrame: DataFrame with lagged variables.
        """
        lagged_data = pd.DataFrame(index=self.data.index)
        
        for lag_length in lag_lengths:
            for feature in self.data.filter(like=prefix).columns:
                lagged_data[f'{feature}_lagged_{lag_length}'] = self.data[feature].shift(lag_length)
        
        return lagged_data

    def calculate_seasonal_aggregations(self, prefix: str) -> pd.DataFrame:
        """
        Calculate seasonal aggregations for the features with the given prefix.

        Parameters:
        - prefix (str): The prefix to filter columns.

        Returns:
        - pd.DataFrame: DataFrame with seasonal aggregations.
        """
        seasonal_aggregations = pd.DataFrame(index=self.data.index)
        
        for feature in self.data.filter(like=prefix).columns:
            seasonal_aggregations[f'{feature}_mean_monthly'] = self.data[feature].groupby(self.data.index.month).transform('mean')
        
        return seasonal_aggregations

    def extract_time_based_features(self) -> pd.DataFrame:
        """
        Extract time-based features such as month and year from the index.

        Returns:
        - pd.DataFrame: DataFrame with time-based features.
        """
        time_features = pd.DataFrame(index=self.data.index)
        time_features['month'] = self.data.index.month
        time_features['year'] = self.data.index.year
        return time_features

    def calculate_expanding_metrics(self, prefix: str) -> pd.DataFrame:
        """
        Calculate expanding metrics for the features with the given prefix.

        Parameters:
        - prefix (str): The prefix to filter columns.

        Returns:
        - pd.DataFrame: DataFrame with expanding metrics.
        """
        expanding_metrics = pd.DataFrame(index=self.data.index)
        
        for feature in self.data.filter(like=prefix).columns:
            expanding_metrics[f'{feature}_expanding_mean'] = self.data[feature].expanding().mean()
            expanding_metrics[f'{feature}_expanding_std'] = self.data[feature].expanding().std()
        
        return expanding_metrics

    def calculate_manual_seasonal_decomposition(self, feature: str, period: int) -> pd.DataFrame:
        """
        Manually calculate seasonal decomposition for a specific feature.

        Parameters:
        - feature (str): The feature to decompose.
        - period (int): The period of the seasonality.

        Returns:
        - pd.DataFrame: DataFrame with trend, seasonal, and residual components.
        """
        data = self.data[feature]
        decomposer = ManualSeasonalDecomposition(data, period)
        components = decomposer.decompose()
        
        decomposition = pd.DataFrame(index=self.data.index)
        decomposition[f'{feature}_trend'] = components['trend']
        decomposition[f'{feature}_seasonal'] = components['seasonal']
        decomposition[f'{feature}_residual'] = components['residual']
        
        return decomposition


# Usage Example
# Assuming `monthly_aggregation` is a pre-defined DataFrame with a DateTime index

target = monthly_aggregation['ΙΣΧΥΣ (MW)_sum']

# Initialize the feature engineering class
engineer = TimeSeriesFeatureEngineering(monthly_aggregation)

# Perform seasonal decomposition for MW features
decomposition_mw = engineer.calculate_manual_seasonal_decomposition('ΙΣΧΥΣ (MW)_sum', period=12)

engineer.data = pd.concat([monthly_aggregation, decomposition_mw], axis=1)

# Calculate autocorrelation for MW features
autocorr_mw = engineer.calculate_autocorrelation(engineer.mw_prefix)

# Create lagged variables for MW features
lagged_data_mw = engineer.create_lagged_variables(engineer.mw_prefix)

# Calculate seasonal aggregations for MW features
seasonal_aggregations_mw = engineer.calculate_seasonal_aggregations(engineer.mw_prefix)

# Calculate expanding metrics for MW features

expanding_metrics_mw = engineer.calculate_expanding_metrics(engineer.mw_prefix)

# Calculate rolling metrics for MW features
rolling_metrics_mw = engineer.calculate_rolling_metrics(engineer.mw_prefix)


# Repeat the process for RSI features

autocorr_rsi = engineer.calculate_autocorrelation(engineer.rsi_prefix)
lagged_data_rsi = engineer.create_lagged_variables(engineer.rsi_prefix)
seasonal_aggregations_rsi = engineer.calculate_seasonal_aggregations(engineer.rsi_prefix)
expanding_metrics_rsi = engineer.calculate_expanding_metrics(engineer.rsi_prefix)
rolling_metrics_rsi = engineer.calculate_rolling_metrics(engineer.rsi_prefix)

# Extract time-based features
time_features = engineer.extract_time_based_features()

# Concatenate all the results into a single DataFrame
final_result_with_target = pd.concat([
    monthly_aggregation, decomposition_mw, 
    rolling_metrics_mw, autocorr_mw, lagged_data_mw, seasonal_aggregations_mw, expanding_metrics_mw,
                          rolling_metrics_rsi, autocorr_rsi, lagged_data_rsi, seasonal_aggregations_rsi, expanding_metrics_rsi,
                          time_features], axis=1)

last_index = final_result_with_target['ΙΣΧΥΣ (MW)_sum'].last_valid_index()

# Concatenate all the results into a single DataFrame
final_result = pd.concat([
    # monthly_aggregation, decomposition_mw, 
    rolling_metrics_mw, autocorr_mw, lagged_data_mw, seasonal_aggregations_mw, expanding_metrics_mw,
                          rolling_metrics_rsi, autocorr_rsi, lagged_data_rsi, seasonal_aggregations_rsi, expanding_metrics_rsi,
                          time_features], axis=1)


/tmp/ipykernel_20830/436709275.py:54: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_20830/436709275.py:54: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_20830/436709275.py:54: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_20830/436709275.py:54: Performance

In [63]:
import plotly.graph_objs as go

# Create a trace for each MW column
traces = []
for column in ['ΙΣΧΥΣ (MW)_count', 'ΙΣΧΥΣ (MW)_sum', 'ΙΣΧΥΣ (MW)_mean', 'ΙΣΧΥΣ (MW)_min', 'ΙΣΧΥΣ (MW)_max', "ΙΣΧΥΣ (MW)_sum_trend", "ΙΣΧΥΣ (MW)_sum_seasonal", "ΙΣΧΥΣ (MW)_sum_residual"]:
    trace = go.Scatter(x=final_result_with_target.index, y=final_result_with_target[column], mode='lines', name=column)
    traces.append(trace)

# Create layout
layout = go.Layout(title='MW Time Series',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='MW'))

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Plot figure
fig.show()


In [64]:
last_index

Timestamp('2024-02-29 00:00:00')

In [65]:
import pandas as pd
import lightgbm as lgb
from datetime import timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Assuming final_result and target are defined elsewhere in your code
df = final_result.copy()
df['ΙΣΧΥΣ (MW)_sum'] = target

# # Ensure the index is in datetime format
# if not pd.api.types.is_datetime64_any_dtype(df.index):
#     df.index = pd.to_datetime(df.index, errors='coerce')

validity_offset = timedelta(days=360)

# Define the last date for the training set
train_end_date = last_index - validity_offset

# Split the data into training and validation sets
train_df = df[df.index <= train_end_date]
valid_df = df[(df.index > train_end_date) & (df.index <= last_index)]
pred_df = df[(df.index > last_index)]

# Features and target
features = df.columns.difference(['ΙΣΧΥΣ (MW)_sum'])
target_lbl = 'ΙΣΧΥΣ (MW)_sum' 

X_train, y_train = train_df[features], train_df[target_lbl]
X_valid, y_valid = valid_df[features], valid_df[target_lbl]
X_pred = pred_df[features]

# LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

# Parameters
params = {
    'objective': 'regression',
    'metric': 'mape',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'early_stopping_rounds': 400
}

# Training
num_boost_round = 1000

bst = lgb.train(
    params,
    train_data,
    num_boost_round=num_boost_round,
    valid_sets=[train_data, valid_data]
)

# Predictions for the validation period
y_valid_pred = bst.predict(X_valid, num_iteration=bst.best_iteration)

# Evaluate RMSE
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'Validation RMSE: {rmse}')

# Evaluate MAPE
mape = mean_absolute_percentage_error(y_valid, y_valid_pred)
print(f'Validation MAPE: {mape}')

y_forecast = bst.predict(X_pred, num_iteration=bst.best_iteration)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020315 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69067
[LightGBM] [Info] Number of data points in the train set: 214, number of used features: 1244
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3135.750318
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 400 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [67]:
# Concatenate actuals and predictions
actuals = df[target_lbl]
valid_predictions = pd.Series(y_valid_pred, index=X_valid.index)
forecast_predictions = pd.Series(y_forecast, index=future_dates)

combined_df = pd.concat([actuals, valid_predictions, forecast_predictions], axis=1)
combined_df.columns = ['Actuals', 'Validation Predictions', 'Forecast Predictions']

# Plot with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=combined_df.index, y=combined_df['Actuals'], mode='lines', name='Actuals'))
fig.add_trace(go.Scatter(x=valid_predictions.index, y=valid_predictions, mode='lines', name='Validation Predictions'))
fig.add_trace(go.Scatter(x=forecast_predictions.index, y=forecast_predictions, mode='lines', name='Forecast Predictions'))

fig.update_layout(
    title='Actuals and Predictions',
    xaxis_title='Date',
    yaxis_title='ΙΣΧΥΣ (MW)_sum',
    template='plotly_white'
)

fig.show()